In [32]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem.porter import PorterStemmer
import pickle


In [33]:
# Load data
movies = pd.read_csv("movies_metadata.csv")
ratings = pd.read_csv("ratings.csv")
credits = pd.read_csv("credits.csv")
keywords_movies = pd.read_csv("keywords.csv")


C:\Users\Dell\AppData\Local\Temp\ipykernel_20844\3154424807.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


In [34]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [35]:
# Data preprocessing
movies = movies.drop(columns=['budget', 'homepage', 'imdb_id', 'popularity', 'production_companies',
                             'production_countries', 'revenue', 'runtime', 'spoken_languages', 'status', 
                             'tagline', 'original_title', 'video', 'vote_average', 'vote_count'])


In [37]:
movies.head(1)

,adult,belongs_to_collection,genres,id,original_language,overview,poster_path,release_date,title
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,Toy Story


In [38]:
movies.isnull().sum()

adult                        0
belongs_to_collection    40972
genres                       0
id                           0
original_language           11
overview                   954
poster_path                386
release_date                87
title                        6
dtype: int64

In [41]:
movies.dropna(inplace=True)
movies["id"] = movies["id"].astype(int)

In [42]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [43]:
ratings = ratings.drop(columns=['timestamp', "userId"])
ratings.drop_duplicates(inplace=True)
ratings.rename(columns={"movieId": "id"}, inplace=True)

In [44]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [45]:
credits.drop_duplicates(inplace=True)

In [47]:
keywords_movies.head(1)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [48]:
keywords_movies['keywords'][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [49]:
keywords_movies.drop_duplicates(inplace=True)

In [50]:
# Merging all dataframes
movies = pd.merge(movies, ratings, on="id")
movies = pd.merge(movies, credits, on='id')
movies = pd.merge(movies, keywords_movies, on='id')

In [51]:
movies.head(1)

,adult,belongs_to_collection,genres,id,original_language,overview,poster_path,release_date,title,rating,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,Toy Story,3.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [52]:
# Remove duplicate rows for the same movie
movies.drop_duplicates(subset=['id'], inplace=True)

In [53]:
# Reset the index to ensure alignment with the similarity matrix
movies.reset_index(drop=True, inplace=True)

In [54]:
# Convert genres, cast, crew, and keywords to lists
def convert(obj):
    if obj and isinstance(obj, str):
        try:
            genre_list = ast.literal_eval(obj)
            if isinstance(genre_list, list):
                return [genre['name'] for genre in genre_list if 'name' in genre]
        except (ValueError, SyntaxError):
            pass
    return []


In [55]:
movies["genres"] = movies["genres"].apply(convert)
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(lambda x: [ast.literal_eval(x)['name']] if pd.notna(x) else [])
movies['cast'] = movies['cast'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)][:3])
movies['crew'] = movies['crew'].apply(lambda x: [i['name'] for i in ast.literal_eval(x) if i['job'] == 'Director'])
movies['keywords'] = movies['keywords'].apply(convert)


In [56]:
# Combine all features into tags
movies['tags'] = (
    movies['belongs_to_collection'] +  # Ensure it's a list
    movies['genres'] +  # Already a list
    movies['overview'].apply(lambda x: x.split()) +  # Split overview into a list of words
    movies['cast'] +  # Already a list
    movies['crew'] +  # Already a list
    movies['keywords']  # Already a list
)

In [57]:
movies.head(1)

,adult,belongs_to_collection,genres,id,original_language,overview,poster_path,release_date,title,rating,cast,crew,keywords,tags
0,False,[Toy Story Collection],"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,Toy Story,3.0,"[Tom Hanks, Tim Allen, Don Rickles]",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva...","[Toy Story Collection, Animation, Comedy, Fami..."


In [58]:
movies['tags'][0]

['Toy Story Collection',
 'Animation',
 'Comedy',
 'Family',
 'Led',
 'by',
 'Woody,',
 "Andy's",
 'toys',
 'live',
 'happily',
 'in',
 'his',
 'room',
 'until',
 "Andy's",
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'the',
 'scene.',
 'Afraid',
 'of',
 'losing',
 'his',
 'place',
 'in',
 "Andy's",
 'heart,',
 'Woody',
 'plots',
 'against',
 'Buzz.',
 'But',
 'when',
 'circumstances',
 'separate',
 'Buzz',
 'and',
 'Woody',
 'from',
 'their',
 'owner,',
 'the',
 'duo',
 'eventually',
 'learns',
 'to',
 'put',
 'aside',
 'their',
 'differences.',
 'Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'John Lasseter',
 'jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy next door',
 'new toy',
 'toy comes to life']

In [59]:
# Converting list of tags into a single string
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [60]:
movies['tags'][0]

"Toy Story Collection Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. Tom Hanks Tim Allen Don Rickles John Lasseter jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life"

In [61]:
# Stemming
ps = PorterStemmer()
def stem(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [62]:
movies['tags'] = movies['tags'].apply(stem)

In [63]:
movies['tags'][0]

"toy stori collect anim comedi famili led by woody, andy' toy live happili in hi room until andy' birthday bring buzz lightyear onto the scene. afraid of lose hi place in andy' heart, woodi plot against buzz. but when circumst separ buzz and woodi from their owner, the duo eventu learn to put asid their differences. tom hank tim allen don rickl john lasset jealousi toy boy friendship friend rivalri boy next door new toy toy come to life"

In [64]:
# Vectorization
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = tfidf.fit_transform(movies['tags']).toarray()

In [65]:
# Calculate similarity
similarity = cosine_similarity(vectors)

In [66]:
# Recommendation function
def recommend(movie):
    # Find the index of the movie
    movie_index = movies[movies['title'] == movie].index[0]
    
    # Get the similarity scores for the movie
    distances = similarity[movie_index]
    
    # Sort the movies based on similarity scores
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Print the top 5 recommended movies
    for i in movie_list:
        print(movies.iloc[i[0]].title)

In [67]:
# Test the recommendation function
recommend('GoldenEye')


You Only Live Twice
The Living Daylights
From Russia with Love
Octopussy
Goldfinger


In [68]:
recommend("Batman")

Batman & Robin
Batman Returns
The Dark Knight
Batman Begins
Batman Forever


In [69]:
# Save the data and similarity matrix
pickle.dump(movies.to_dict(), open('movie_dict.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))